In [ ]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs236299-2020/lab4-3.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

%%latex
\newcommand{\vect}[1]{\mathbf{#1}}
\newcommand{\cnt}[1]{\sharp(#1)}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\softmax}{\operatorname{softmax}}
\newcommand{\Prob}{\Pr}
\newcommand{\given}{\,|\,}

$$
\renewcommand{\vect}[1]{\mathbf{#1}}
\renewcommand{\cnt}[1]{\sharp(#1)}
\renewcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\renewcommand{\softmax}{\operatorname{softmax}}
\renewcommand{\Prob}{\Pr}
\renewcommand{\given}{\,|\,}
$$

# Lab 4-3 - Sequence-to-sequence models

In lab 4-2, we used a syntactic-semantic grammar for semantic parsing to convert natural language into meaning representations in SQL. In this lab, we consider an alternative approach - sequence-to-sequence models ("seq2seq"). Seq2seq models address this task directly, by learning the mapping from a sequence of inputs to a sequence of outputs. Since seq2seq models make only few assumptions about the data, they can be applied to a variety of tasks, including machine translation, document summarization, speech recognition, and more. 

In this lab, we will implement a seq2seq model in its most basic form (as in [this seminal paper by Sutskever et al., 2014](https://arxiv.org/pdf/1409.3215.pdf)), and apply it to the task of words-to-numbers conversion, as shown in the table below.

|                 Input                                            |       Output         |
|------------------------------------------------------------------|----------------------|
| seven thousand nine hundred and twenty nine                      |       7929           |
| eight hundred and forty two thousand two hundred and fifty nine  |       842259         |
| five hundred and eight thousand two hundred and seventeen        |       508217         |

For this simple task, it is possible to write a rule-based program to do the conversion. However, here we take a learning-based approach and learn the mapping from examples, with the benefit that the architecture that we implement here can be trained for other seq2seq tasks as well (including the ATIS-to-SQL problem in Homework 4).

## Pytorch functions used in solution code

* [torch.transpose](https://pytorch.org/docs/stable/generated/torch.transpose.html): swap two dimensions of a tensor
* [torch.reshape](https://pytorch.org/docs/stable/generated/torch.reshape.html): reshape a tensor
* [torch.nn.utils.rnn.pack_padded_sequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html): (imported as `pack`) handles paddings. A more detailed explanation can be found at [here](https://stackoverflow.com/a/56211056)

## Preparation - Loading data

In [ ]:
import math
import warnings
import copy

import torch
import torch.nn as nn
import torchtext as tt

from tqdm import tqdm

from torch.nn.utils.rnn import pack_padded_sequence as pack

In [ ]:
# GPU check, make sure to use GPU where available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

## Turn off annoying torchtext warnings about pending deprecations
warnings.filterwarnings("ignore", module="torchtext", category=UserWarning)

cuda


In [ ]:
# Download data
shell("""
  wget -nv -N -P data https://raw.githubusercontent.com/nlp-236299/data/master/Words2Num/train.src
  wget -nv -N -P data https://raw.githubusercontent.com/nlp-236299/data/master/Words2Num/train.tgt
  wget -nv -N -P data https://raw.githubusercontent.com/nlp-236299/data/master/Words2Num/dev.src
  wget -nv -N -P data https://raw.githubusercontent.com/nlp-236299/data/master/Words2Num/dev.tgt
  wget -nv -N -P data https://raw.githubusercontent.com/nlp-236299/data/master/Words2Num/test.src
  wget -nv -N -P data https://raw.githubusercontent.com/nlp-236299/data/master/Words2Num/test.tgt
""")

First, let's take a look at the dataset.

In [ ]:
with open('data/dev.src') as fsrc:
    with open('data/dev.tgt') as ftgt:
        print (f'{"Source":70s} {"Target":10s}')
        for src, tgt, _ in zip(fsrc, ftgt, range(3)):
            print (f'{src.strip():70s} {tgt.strip():10s}')

Source                                                                 Target    
seven thousand nine hundred and twenty nine                            7 9 2 9   
eight hundred and forty two thousand two hundred and fifty nine        8 4 2 2 5 9
five hundred and eight thousand two hundred and seventeen              5 0 8 2 1 7


As before, we use `torchtext` to load data. We use two fields: `SRC` for processing the source side (the words) and `TGT` for processing the target side (the numbers).

In [ ]:
SRC = tt.data.Field(include_lengths=True,         # include lengths
                    batch_first=False,            # batches will be max_len x batch_size
                    tokenize=lambda x: x.split(), # use split to tokenize
                   ) 
TGT = tt.data.Field(include_lengths=False,
                    batch_first=False,            # batches will be max_len x batch_size
                    tokenize=lambda x: x.split(), # use split to tokenize
                    init_token="<bos>",           # prepend <bos>
                    eos_token="<eos>")            # append <eos>
fields = [('src', SRC), ('tgt', TGT)]

Note that we prepended `<bos>` and appended `<eos>` to target sentences. The purpose for introducing them will become clear in later parts of this lab.

In [ ]:
# Make splits for data
train_data, val_data, test_data = tt.datasets.TranslationDataset.splits(
    ('.src', '.tgt'), fields, path='./data',
    train='train', validation='dev', test='test')

# Build vocabulary
SRC.build_vocab(train_data.src)
TGT.build_vocab(train_data.tgt)

print (f"Size of src vocab: {len(SRC.vocab)}")
print (f"Size of tgt vocab: {len(TGT.vocab)}")
print (f"Index for src padding: {SRC.vocab.stoi[SRC.pad_token]}")
print (f"Index for tgt padding: {TGT.vocab.stoi[TGT.pad_token]}")
print (f"Index for start of sequence token: {TGT.vocab.stoi[TGT.init_token]}")
print (f"Index for end of sequence token: {TGT.vocab.stoi[TGT.eos_token]}")

Size of src vocab: 34
Size of tgt vocab: 14
Index for src padding: 1
Index for tgt padding: 1
Index for start of sequence token: 2
Index for end of sequence token: 3


We batch training and validation data into minibatches, but for the test set, we use a batch size of 1, to make decoding implementation easier.

In [ ]:
BATCH_SIZE = 32     # batch size for training and validation
TEST_BATCH_SIZE = 1 # batch size for test; we use 1 to make implementation easier

train_iter, val_iter = tt.data.BucketIterator.splits((train_data, val_data),
                                                     batch_size=BATCH_SIZE, 
                                                     device=device,
                                                     repeat=False, 
                                                     sort_key=lambda x: len(x.src), # sort by length to minimize padding
                                                     sort_within_batch=True)
test_iter = tt.data.BucketIterator(test_data, 
                                   batch_size=TEST_BATCH_SIZE, 
                                   device=device,
                                   repeat=False, 
                                   sort=False, 
                                   train=False)

Let's take a look at a batch from these iterators.

In [ ]:
batch = next(iter(train_iter))
src, src_lengths = batch.src
print (f"Size of src batch: {src.shape}")
print (f"Third src sentence in batch: {src[:, 2]}")
print (f"Length of the third src sentence in batch: {src_lengths[2]}")
print (f"Converted back to string: {' '.join([SRC.vocab.itos[i] for i in src[:, 2]])}")

tgt = batch.tgt
print (f"Size of tgt batch: {tgt.shape}")
print (f"Third tgt sentence in batch: {tgt[:, 2]}")
print (f"Converted back to string: {' '.join([TGT.vocab.itos[i] for i in tgt[:, 2]])}")

Size of src batch: torch.Size([8, 32])
Third src sentence in batch: tensor([21, 11,  4, 10,  3,  2, 19, 14], device='cuda:0')
Length of the third src sentence in batch: 8
Converted back to string: twenty six thousand nine hundred and eighty four
Size of tgt batch: torch.Size([8, 32])
Third tgt sentence in batch: tensor([ 2,  7,  8,  5, 11, 10,  3,  1], device='cuda:0')
Converted back to string: <bos> 2 6 9 8 4 <eos> <pad>


## Neural Encoder-Decoder Models

Sequence-to-sequence models are sometimes called neural encoder-decoder models, as they consist of an encoder, which maps a sequence of source tokens into some vector representations, and a decoder, which generates a sequence of output words using those encoded vectors.

Formally, given a sequence of source tokens $\vect{x} = x_1, \ldots, x_S$, the goal is to map it to a sequence of target tokens $\vect{y} = y_1, \ldots, y_T$. 

In practice, we prepend a special beginning-of-sequence symbol $y_0$=`<bos>` to the target sequence. Further, in order to provide a way of knowing when to stop generating $\vect{y}$, we append a special end-of-sequence symbol $y_{T+1}$=`<eos>` to the target sequence, such that when it is produced by the model, the generation process stops.

The generation process is structured as a generative model:

$$
P(y_0, \ldots, y_{T+1} \mid x_1, \ldots, x_S) = \prod_{t=1}^{T+1} P(y_t \mid y_{<t}, x_1, \ldots, x_S),
$$

where $y_{<t}$ denotes the tokens before $y_t$ (that is, $y_0, \ldots, y_{t-1}$).

We use a recurrent neural network with parameters $\theta$ to parameterize $P(y_t \mid y_{<t}, x_1, \ldots, x_S)$:

$$
P(y_t \mid y_{<t}, x_1, \ldots, x_S) \approx P_\theta(y_t \mid y_{<t}, x_1, \ldots, x_S),
$$

or equivalently,

$$
P_\theta(y_1, \ldots, y_T \mid x_1, \ldots, x_S) = \prod_{t=1}^{T+1} P_\theta(y_t \mid y_{<t}, x_1, \ldots, x_S)
$$

In neural encoder-decoder models, we first use an encoder to encode $\vect{x}$ into some vectors (either of fixed length as we'll see in this lab, or of varying length as we'll see in the next lab). Based on the encoded vectors, we use a decoder to generate $\vect{y}$:

$$
P_\theta(y_t \mid y_{<t}, x_1, \ldots, x_S) = \text{decode}(\text{encode}(x_1, \ldots, x_S), y_{<t})
$$

### RNN Encoder-Decoders

We can use any recurrent neural networks such as LSTMs as encoders and decoders. In this lab, we will use a **bidirectional** LSTM as the **encoder**, and a **unidirectional** LSTM as the **decoder**, as shown in the illustration below.

<img src="https://github.com/nlp-236299/data/raw/master/img/encoder_decoder.png" alt="encoder-decoder illustration" />

In the above illustration, $S=4$, $T=3$, and there are two encoder/decoder layers. Since we are using a bidirectional encoder, for each layer there are two final states, one for the cell running from left to right (such as $h_{0,4}$), and the other for the cell running from right to left (such as $h_{0,4}'$). We concatenate these two states and use the result to initialize the corresponding layer of the decoder. (In the example, we concatenate $h_{0,4}$ and $h_{0,4}'$ to initialize layer 0, and we concatenate $h_{1,4}$ and $h_{1,4}'$ to initialize layer 1.) Therefore, to make sizes match, we set the hidden state size of the encoder to be half of that of the decoder.

Note that in PyTorch's LSTM implementation, the final hidden state is represented as a tuple `(h, c)` ([documentation here](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)), so we want to apply the same operations to `c` to initialize the decoder.

You'll implement `forward_encoder`, `forward_decoder`, and `forward` in the code below. The `forward_encoder` function will be reminiscent of a sequence model from labs 2-* and homework 2. It operates on a batch of source examples and proceeds as follows:

1. Map the input words to some word embeddings. You'll notice that the embedding size is an argument to the model.

2. Optionally "pack" the sequences to save some computation using `torch.nn.utils.rnn.pack_padded_sequence`, imported above as `pack`.

3. Run the encoder RNN (a bidirectional LSTM) over the batch, generating a batch of output states.

4. Reshape/concatenate the final state information (which will have `h` and `c` components each of half the size needed to initialize the decoder) so it is appropriate to initialize the decoder with.

The `forward_decoder` function takes the reshaped encoder final state information and the ground truth target sequences and returns logits (unnormalized log probs) for each target word. These logits are ready to be converted to probability distributions via a softmax.

The steps in decoding are:

1. Map the target words to word embeddings.

2. Run the decoder RNN (a unidirectional LSTM) over the batch, initializing the hidden units from the encoder final states, generating a batch of output states.

3. Map the RNN outputs to vectors of vocabulary size (so that they could be softmxed into a distribution over the vocabulary).

The `forward` function runs the encoder to get the final state information and then, using this final state, runs the decoder to get the logits. 

The components that you'll be plugging together to do all this are already established in the `__init__` method. 

> The major exception is the reshaping of the encoder output `h` and `c` to form the decoder input `h` and `c`. That's the trickiest part. As usual, your best strategy is to keep careful track of the shapes of each input and output of a layer or operation.

**Hint #1:** You do not need to use *any* `for` loops.

**Hint #2:** According to the documentation of [torch.nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html), its outputs are: `outputs, (h, c)`. `output` contains all the intermediate states, which you don't need in this lab. You will need `h` and `c`, both of them have the shape: `(num_layers * num_directions, batch_size, hidden_size)`.

<!--
BEGIN QUESTION
name: encoder_decoder
-->

In [ ]:

#TODO - finish implementing the `forward_encoder` and `forward_decoder` methods
class EncoderDecoder(nn.Module):
  def __init__(self, src_field, tgt_field, embedding_size=64, hidden_size=64, layers=3):
    """
    Initializer. Creates network modules and loss function.
    Arguments:
        src_field: src field
        tgt_field: tgt field
        embedding_size: word embedding size
        hidden_size: hidden layer size of both encoder and decoder
        layers: number of layers of both encoder and decoder
    """
    super(EncoderDecoder, self).__init__()
    self.src_field = src_field
    self.tgt_field = tgt_field
    
    # Keep the vocabulary sizes available
    self.V_src = len(src_field.vocab.itos)
    self.V_tgt = len(tgt_field.vocab.itos)
    
    # Get special word ids or tokens
    self.padding_id_src = src_field.vocab.stoi[src_field.pad_token]
    self.padding_id_tgt = tgt_field.vocab.stoi[tgt_field.pad_token]
    self.bos_id = tgt_field.vocab.stoi[tgt_field.init_token]
    self.eos_id = tgt_field.vocab.stoi[tgt_field.eos_token]

    # Keep hyper-parameters available
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.layers = layers

    # Create essential modules
    self.word_embeddings_src = nn.Embedding(self.V_src, embedding_size)
    self.word_embeddings_tgt = nn.Embedding(self.V_tgt, embedding_size)

    # RNN cells
    self.encoder_rnn = nn.LSTM(
      input_size    = embedding_size,
      hidden_size   = hidden_size // 2, # to match decoder hidden size
      num_layers    = layers,
      bidirectional = True              # bidirectional encoder
    )
    self.decoder_rnn = nn.LSTM(
      input_size    = embedding_size,
      hidden_size   = hidden_size,
      num_layers    = layers,
      bidirectional = False             # unidirectional decoder
    )

    # Final projection layer
    self.hidden2output = nn.Linear(hidden_size, self.V_tgt)
   
    # Create loss function
    self.loss_function = nn.CrossEntropyLoss(reduction='sum', 
                                             ignore_index=self.padding_id_tgt)

  def forward_encoder(self, src, src_lengths):
    """
    Encodes source words `src`.
    Arguments:
        src: src batch of size (max_src_len, batch_size)
        src_lengths: src lengths of size (batch_size)
    Returns:
        a tuple (h, c) where h/c is of size (layers, batch_size, hidden_size) 
    """
    # Compute word embeddings
    src_embeddings = self.word_embeddings_src(src) # max_len, batch_size, embedding_size
    src_lengths = src_lengths.tolist()
    # Deal with paddings
    packed_src = pack(src_embeddings, src_lengths)
    # TODO - implement this function

    _, (h,c) = self.encoder_rnn(packed_src)
    shape = h.shape
    h = h.view(2, int(shape[0]/2), shape[1], shape[2])
    h = torch.cat([h[0], h[1]], dim=2)
    c = c.view(2, int(shape[0]/2), shape[1], shape[2])
    c = torch.cat([c[0], c[1]], dim=2)
    return h,c

  def forward_decoder(self, encoder_final_state, tgt_in):
    """
    Decodes based on encoder final state and ground truth target words.
    Arguments:
        encoder_final_state: a tuple (h, c) where h/c is of size (layers, bsz, hidden_size)
        tgt_in: a tensor of size (tgt_len, bsz)
    Returns:
        Logits of size (tgt_len, bsz, V_tgt) (before the softmax operation)
    """
    # TODO - implement this function
    tgt_embeddings = self.word_embeddings_tgt(tgt_in) # max_len, batch_size, embedding_size
    #tgt_lengths = tgt_lengths.tolist()

    #packed_src = pack(tgt_embeddings, tgt_lengths)

    #seq_unpacked, lens_unpacked = pad_packed_sequence(encoder_final_state, batch_first=True)

    h,c = self.decoder_rnn(tgt_embeddings, encoder_final_state)

    logits = self.hidden2output(h)

    return logits

  def forward(self, src, src_lengths, tgt_in):
    """
    Performs forward computation, returns logits.
    Arguments:
        src: src batch of size (max_src_len, batch_size)
        src_lengths: src lengths of size (batch_size)
        tgt_in:  a tensor of size (tgt_len, bsz)
    """
    h_c = self.forward_encoder(src, src_lengths)
    logits = self.forward_decoder(h_c, tgt_in)
    return logits

  def forward_decoder_incrementally(self, decoder_state, tgt_in_token):
    """
    Forward the decoder at `decoder_state` for a single step with token `tgt_in_token`.
    This function will only be used in the beam search section.
    Arguments:
        encoder_final_state: a tuple (h, c) where h/c is of size (layers, 1, hidden_size)
        tgt_in_token: a tensor of size (1), a single token
    Returns:
        `logits`: Log probabilities for `tgt_in_token` of size (V_tgt)
        `decoder_state`: updated decoder state, ready for next incremental update
    """
    bsz = decoder_state[0].size(1)
    assert bsz == 1, "forward_decoder_incrementally only supports batch size 1!"
    # Compute word embeddings
    tgt_embeddings = self.word_embeddings_tgt(tgt_in_token.view(1, 1)) # tgt_len, bsz, hidden
    # Forward decoder RNN and return all hidden states
    decoder_outs, decoder_state = self.decoder_rnn(tgt_embeddings, decoder_state)
    # Project to get logits
    logits = self.hidden2output(decoder_outs) # tgt_len, bsz, V_tgt
    # Get log probabilities
    logits = torch.log_softmax(logits, -1)
    return logits.view(-1), decoder_state

  def evaluate_ppl(self, iterator):
    """Returns the model's perplexity on a given dataset `iterator`."""
    # Switch to eval mode
    self.eval()
    total_loss = 0
    total_words = 0
    for batch in iterator:
      # Input and target
      src, src_lengths = batch.src
      tgt = batch.tgt # max_length_sql, bsz
      tgt_in = tgt[:-1] # remove <eos> for decode input (y_0=<bos>, y_1, y_2)
      tgt_out = tgt[1:] # remove <bos> as target        (y_1, y_2, y_3=<eos>)
      # Forward to get logits
      logits = self.forward(src, src_lengths, tgt_in)
      # Compute cross entropy loss
      loss = self.loss_function(logits.view(-1, self.V_tgt), tgt_out.view(-1))
      total_loss += loss.item()
      total_words += tgt_out.ne(self.padding_id_tgt).float().sum().item()
    return math.exp(total_loss/total_words)

  def train_all(self, train_iter, val_iter, epochs=10, learning_rate=0.001):
    """Train the model."""
    # Switch the module to training mode
    self.train()
    # Use Adam to optimize the parameters
    optim = torch.optim.Adam(self.parameters(), lr=learning_rate)
    best_validation_ppl = float('inf')
    best_model = None
    # Run the optimization for multiple epochs
    for epoch in range(epochs): 
      total_words = 0
      total_loss = 0.0
      for batch in tqdm(train_iter):
        # Zero the parameter gradients
        self.zero_grad()
        # Input and target
        src, src_lengths = batch.src # text: max_src_length, bsz
        tgt = batch.tgt # max_tgt_length, bsz
        tgt_in = tgt[:-1] # Remove <eos> for decode input (y_0=<bos>, y_1, y_2)
        tgt_out = tgt[1:] # Remove <bos> as target        (y_1, y_2, y_3=<eos>)
        batch_size = tgt.size(1)
        # Run forward pass and compute loss along the way.
        logits = self.forward(src, src_lengths, tgt_in)
        loss = self.loss_function(logits.view(-1, self.V_tgt), tgt_out.view(-1))
        # Training stats
        num_tgt_words = tgt_out.ne(self.padding_id_tgt).float().sum().item()
        total_words += num_tgt_words
        total_loss += loss.item()
        # Perform backpropagation
        loss.div(batch_size).backward()
        optim.step()

      # Evaluate and track improvements on the validation dataset
      validation_ppl = self.evaluate_ppl(val_iter)
      self.train()
      if validation_ppl < best_validation_ppl:
        best_validation_ppl = validation_ppl
        self.best_model = copy.deepcopy(self.state_dict())
      epoch_loss = total_loss / total_words
      print (f'Epoch: {epoch} Training Perplexity: {math.exp(epoch_loss):.4f} '
             f'Validation Perplexity: {validation_ppl:.4f}')


In [ ]:
EPOCHS = 2 # epochs, we highly recommend starting with a smaller number like 1
LEARNING_RATE = 2e-3 # learning rate

# Instantiate and train classifier
model = EncoderDecoder(SRC, TGT,
  embedding_size = 64,
  hidden_size    = 64,
  layers         = 3,
).to(device)

model.train_all(train_iter, val_iter, epochs=EPOCHS, learning_rate=LEARNING_RATE)
model.load_state_dict(model.best_model)

  0%|          | 3/2032 [00:00<01:10, 28.61it/s]

Epoch: 0 Training Perplexity: 1.7406 Validation Perplexity: 1.1009


100%|██████████| 2032/2032 [00:29<00:00, 69.83it/s]


Epoch: 1 Training Perplexity: 1.0610 Validation Perplexity: 1.0221


<All keys matched successfully>

Since the task we consider here is very simple, we should expect a perplexity very close to 1.

In [ ]:
# Evaluate model performance, the expected value should be < 1.6
print (f'Test perplexity: {model.evaluate_ppl(test_iter):.3f}')

Test perplexity: 1.031


In [ ]:
grader.check("encoder_decoder_ppl")

All tests passed!

<!-- BEGIN QUESTION -->

## Decoding

Now that we have a well-trained model, we need to consider how to use it to do the actual conversion. At decoding time, given a source sequence $x_1, \ldots, x_S$, we want to find the target sequence $y_1^*, \ldots, y_T^*, y_{T+1}^*$ (recall that $y _{T+1} = \langle\text{eos}\rangle$) such that the conditional likelihood is maximized:

\begin{align*}
y_1^*, \ldots, y_T^*, y_{T+1}^* &= \argmax{y_1, \ldots, y_T, y_{T+1}} P_\theta(y_1, \ldots, y_T \mid x_1, \ldots, x_S)\\
&= \argmax{y_1, \ldots, y_T, y_{T+1}} \prod_{t=1}^{T+1} P_\theta(y_t \mid y_{<t}, x_1, \ldots, x_S)\\
\end{align*}

In previous labs and project segments, we used _greedy decoding_, i.e., taking $\hat{y}_1= \argmax{y_1} P_\theta(y_1 \mid y_{0}, x_1, \ldots, x_S)$, $\hat{y}_2= \argmax{y_2} P_\theta(y_2 \mid y_{0}, \hat{y}_1, x_1, \ldots, x_S)$, ..., $\hat{y}_{T+1}= \argmax{y_{T+1}} P_\theta(y_{T+1} \mid y_{0}, \hat{y}_1, \ldots, \hat{y}_{T}, x_1, \ldots, x_S)$, until $\hat{y}_{T+1}=\langle\text{eos}\rangle$. 

---
**Question:**  Does greedy decoding guarantee finding the optimal sequence (the sequence with the highest conditional likelihood)? Why or why not?

<!--
BEGIN QUESTION
name: open_response_decoding
manual: true
-->

The greedy policy doesn't guarantee finding the optimal sequence, because according to the greedy policy in ome step of the sequence predicting if we choose the one output that maximizes the current score, it can be not the one which maximizes the whole seuence probability, because each output is the next input, thus there could be a step in which if we will choose another sequnce output which has not the maximum probability, we will end in another chain of seequnce outputs which will increase the whole probability.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

---

### Beam search decoding

Beam search decoding is the most commonly used decoding method in sequence-to-sequence approaches. Like greedy decoding, it uses a left-to-right search process. But instead of only keeping the single argmax at each position, beam search maintains the $K$ best partial hypotheses $H_t = \{(y_1^{(k)}, \ldots, y_t^{(k)}): k\in\{1, \ldots, K\}\}$ at every step $t$. To proceed to $t+1$, we compute the scores of sequences  $y_1^{(k)}, \ldots, y_t^{(k)}, y_{t+1}$ for every possible extension $y_{t+1}\in \mathcal{V}$ and every possible prefix $(y_1^{(k)}, \ldots, y_t^{(k)}) \in H_t$, where $\mathcal{V}$ is the vocabulary. Among these $K\times |\mathcal{V}|$ sequences, we only keep the top $K$ sequences with the best partial scores, and that becomes $H_{t+1} = \{(y_1^{(k)}, \ldots, y_{t+1}^{(k)}): k\in\{1, \ldots, K\}\}$. To start at $t=1$, $H_1 = \{(y): y \in \text{K argmax}_{y_1\in\mathcal{V}} \log P(y_1|y_0=bos) \}$. Here $K$ is called the beam size.

To summarize,

\begin{align*}
&H_1 = \{(y): y \in \operatorname*{K argmax}_{y_1\in\mathcal{V}} \log P(y_1|y_0=bos) \}\\
&H_{t+1} = \operatorname*{K argmax}\limits_{\{(y_1, y_2, \ldots, y_{t+1})\in \mathcal{V}^{t+1}: (y_1, \ldots, y_t) \in H_t \}} \log P(y_1, \ldots, y_{t+1} | x)
\end{align*}

until we reach a pre-specified maximum search length, and we collect the completed hypotheses along the way (by completed we mean ending with `<eos>`). The finished hypothesis with the best score will be returned.

---
**Question:** Is beam search better than greedy search when $K=1$? Is it better when $K>1$? Why? How big a $K$ value do we need to get a guarantee that we can find the globally best sequence (assuming max sequence length $T$ and vocabulary size $|\mathcal{V}|$).

<!--
BEGIN QUESTION
name: open_response_beam_search_greedy
manual: true
-->

when K=1 it is the greedy policy which we discussed in the previous question. It is better for K>1 because as we mentioned in the previous question, for K>1 we could find a concrete step where it is more beneficial to choose an output with lesser probability but which will serve as an input to the next step whith bigger probabilities. So as bigger the K, as more combinations of possible outputs we can examine for finding the maximum between them. To find the globally best sequence we could choose K=|V| to check all the possible outcomes of each prediction consecutive step.  

<!-- END QUESTION -->



---
Under the probabilistic formulation of sequence-to-sequence models, the partial scores are decomposable over time steps: $\log P_\theta(y_1, \ldots, y_T \mid x) = \sum_{t=1}^T \log P_\theta(y_t \mid y_{<t}, x)$. Therefore, we can save computation in the above process by maintaining the partial sums $\sum_{t'=1}^t \log P_\theta(y_{t'}^{(k)} \mid y_{<t'}^{(k)}, x)$, such that we only need to compute $\log P_\theta(y_{t+1} \mid y_{<t+1}^{(k)})$ when we want to go from $t$ to $t+1$. 

Here is pseudo-code for the beam search algorithm to decode a single example `x` of maximum length `max_T` using a beam size of `K`.

```
 1.  def beam_search(x, K, max_T):
 2.      finished = []       # for storing completed hypotheses
         # Initialize the beam
 3.      beams = [Beam(hyp=(bos), score=0)] # initial hypothesis: bos, initial score: 0
 
 4.      for t in [1..max_T]  # main body of search over time steps           
 5.          hypotheses = []
 
             # Expand each beam by all possible tokens y_{t+1}
 6.          for beam in beams:
 7.              y_{1:t}, score = beam.hyp, beam.score
 8.              for y_{t+1} in V:
 9.                  y_{1:t+1} = y_{1:t} + [y_{t+1}]
 10.                 new_score = score + log P(y_{t+1} | y_{1:t}, x)
 11.                 hypotheses.append(Beam(hyp=y_{1:t+1}, score=new_score))
 
             # Find K best next beams
 12.         beams = sorted(hypotheses, key=lambda beam: -beam.score)[:K]
 
             # Set aside finished beams (those that end in <eos>)
 13.         for beam in beams:
 14.             y_{t+1} = beam.hyp[-1]
 15.             if y_{t+1} == eos:
 16.                 finished.append(beam)
 17.                 beams.remove(beam)

             # Break the loop if everything is finished
 18.         if len(beams) == 0:
 19.             break              
 20.     return sorted(finished, key=lambda beam: -beam.score)[0] # return the best finished hypothesis
```


Implement function `beam_search` in the below code. Note that there are some differences from the pseudo-code: first, we maintained a `decoder_state` in addition to $y_{1:t}$ and score such that we can compute $P(y_{t+1} \mid y_{<t+1},x)$ efficiently; second, instead of creating a list of actual hypotheses as in lines 8-11, we use tensors to get pointers to the beam id and $y_{t+1}$ that are among the best K next beams.

<!--
BEGIN QUESTION
name: beam_search
-->

In [ ]:
# max target length
MAX_T = 15
class Beam():
    """
    Helper class for storing a hypothesis, its score and its decoder hidden state.
    """

    def __init__(self, decoder_state, tokens, score):
        self.decoder_state = decoder_state
        self.tokens = tokens
        self.score = score


class BeamSearcher():
    """
    Main class for beam search.
    """

    def __init__(self, model):
        self.model = model
        self.bos_id = model.bos_id
        self.eos_id = model.eos_id
        self.V = model.V_tgt

    def beam_search(self, src, src_lengths, K, max_T=MAX_T):
        """
        Performs beam search decoding.
        Arguments:
            src: src batch of size (max_src_len, 1)
            src_lengths: src lengths of size (1)
            K: beam size
            max_T: max possible target length considered
        Returns:
            a list of token ids
        """
        finished = []
        # Initialize the beam
        self.model.eval()
        # TODO - fill in encoder_final_state and init_beam below
        encoder_final_state = self.model.forward_encoder(src, src_lengths)
        bos = torch.tensor([[self.model.bos_id]], device=device)
        init_beam = Beam(encoder_final_state, bos, 0)
        beams = [init_beam]

        for t in range(max_T):  # main body of search over time steps

            # Expand each beam by all possible tokens y_{t+1}
            all_total_scores = []
            for beam in beams:
                y_1_to_t, score, decoder_state = beam.tokens, beam.score, beam.decoder_state
                y_t = y_1_to_t[-1]
                # TODO - finish the code below
                # Hint: you might want to use `model.forward_decoder_incrementally`
                total_scores, decoder_state = self.model.forward_decoder_incrementally(decoder_state, y_t)
                all_total_scores.append(total_scores)
                beam.decoder_state = decoder_state  # update decoder state in the beam
            all_total_scores = torch.stack(all_total_scores)  # (K, V) when t>0, (1, V) when t=0

            # Find K best next beams
            # The below code has the same functionality as line 6-12, but is more efficient
            all_scores_flattened = all_total_scores.view(-1)  # K*V when t>0, 1*V when t=0
            topk_scores, topk_ids = all_scores_flattened.topk(K, 0)
            beam_ids = topk_ids // self.V
            next_tokens = topk_ids - beam_ids * self.V
            new_beams = []
            for k in range(K):
                beam_id = beam_ids[k]  # which beam it comes from
                y_t_plus_1 = next_tokens[k]  # which y_{t+1}
                score = topk_scores[k]
                beam = beams[beam_id]
                decoder_state = beam.decoder_state
                y_1_to_t = beam.tokens
                # TODO
                y_t = torch.cat((y_1_to_t, y_t_plus_1.view(1,1)))
                new_beam = Beam(decoder_state, y_t, beam.score+score)
                new_beams.append(new_beam)
            beams = new_beams

            # Set aside completed beams
            # TODO - move completed beams to `finished` (and remove them from `beams`)
            new_beams = []
            for beam in beams:
                y_1_to_t, score, decoder_state = beam.tokens, beam.score, beam.decoder_state
                y_t = y_1_to_t[-1]
                if y_t.view(1) == self.model.eos_id:
                    finished.append(beam)
                else:
                    new_beams.append(beam)
            beams = new_beams
            # Break the loop if everything is completed
            if len(beams) == 0:
                break

        # Return the best hypothesis
        if len(finished) > 0:
            finished = sorted(finished, key=lambda beam: -beam.score)
            return finished[0].tokens
        else:  # when nothing is finished, return an unfinished hypothesis
            return beams[0].tokens

In [ ]:
grader.check("beam_search")

All tests passed!

Now we can use beam search decoding to predict the outputs for the test set inputs using the trained model.

In [ ]:
DEBUG = True # set to False to disable printing predictions
K = 5 # beam size 5

correct = 0
total = 0

# create beam searcher
beam_searcher = BeamSearcher(model)

for batch in test_iter:
  # Input and output
  src, src_lengths = batch.src
  # Predict
  prediction = beam_searcher.beam_search(src, src_lengths, K)
  # Convert to string
  prediction = ' '.join([TGT.vocab.itos[token] for token in prediction])
  prediction = prediction.lstrip('<bos>').rstrip('<eos>').strip()
  ground_truth = ' '.join([TGT.vocab.itos[token] for token in batch.tgt.view(-1)])
  ground_truth = ground_truth.lstrip('<bos>').rstrip('<eos>').strip()
  if DEBUG:
    src = ' '.join([SRC.vocab.itos[item] for item in src.view(-1)])
    print (f'Source: {src}')
    print (f'Prediction:   {prediction}')
    print (f'Ground truth: {ground_truth}')
  if ground_truth == prediction:
    correct += 1
  total += 1

print (f'Accuracy: {correct/total:.2f}')

Source: sixteen thousand eight hundred and thirty two
Prediction:   1 6 8 3 2
Ground truth: 1 6 8 3 2
Source: sixty seven million six hundred and eighty five thousand two hundred and thirty
Prediction:   6 7 6 8 5 2 3 0
Ground truth: 6 7 6 8 5 2 3 0
Source: six thousand two hundred and twelve
Prediction:   6 2 1 2
Ground truth: 6 2 1 2
Source: seven hundred and ninety eight million three hundred and thirty one thousand eight hundred and eighteen
Prediction:   7 9 8 3 3 1 8 1 8
Ground truth: 7 9 8 3 3 1 8 1 8
Source: eighty eight million four hundred and thirteen thousand nine hundred and eighteen
Prediction:   8 8 4 1 3 9 1 8
Ground truth: 8 8 4 1 3 9 1 8
Source: three hundred and seventy four thousand two hundred and seventy
Prediction:   3 7 4 2 7 0
Ground truth: 3 7 4 2 7 0
Source: ninety eight million three hundred and seventy thousand five hundred and forty five
Prediction:   9 8 3 7 0 5 4 5
Ground truth: 9 8 3 7 0 5 4 5
Source: ninety seven thousand seven hundred and sixty two
Pr

You might have noticed that using a larger $K$ might lead to very similar performance as using $K=1$ (greedy decoding). This is largely due to the fact that there are no dependencies among target tokens in our dataset (e.g., knowing that $y_1$ is 1 does not affect our prediction on $y_2$ conditioned on the source). In real world applications, people usually find using a fixed value of $K>1$ (such as $K=5$) performs better than greedy decoding.

<!-- BEGIN QUESTION -->

---

**Question:** Can we use beam search decoding to decode an HMM? For state space $Q$, sequence length $T$, what is the complexity of beam search with beam size $K$? What is the complexity of Viterbi decoding? What are their pros and cons?

<!--
BEGIN QUESTION
name: open_response_decoding_hmm
manual: true
-->

if we use the beam search, we should consider that the beam search is a greedy algorithm and its main con is that it may not find the best score vs the Viterbi algorithm. Although Viterbi algorithm can be very expenssive, thus take too much time to expend all the pathes, thus in those aspects beam search could help in cutting some pathes accordingly to K.

The beam search on Viterbi variation will work in the following way:
Instead of keeping the entire column of states at each point, we will
keep the best Khypothesis at this point.

The complete complexity of the Veterbi algorithm is O(T x Q^2), where each iteration takes O(Q) complexity. The complexity of beam search with beam size K in this variation of Viterbi will be consequently : O(T x K x Q), because now each iteration will expand furtherly only K states, but we still run on the whole T x Q table. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

---

## Lab debrief – for consensus submission only

**Question:** We're interested in any thoughts your group has about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on include the following: 

* Was the lab too long or too short?
* Were the readings appropriate for the lab? 
* Was it clear (at least after you completed the lab) what the points of the exercises were? 
* Are there additions or changes you think would make the lab better?

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# End of Lab 4-3

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

beam_search:

    All tests passed!
    

encoder_decoder_ppl:

    All tests passed!